In [ ]:
pip list

# 1. Import and Install Dependencies

In [1]:
# !pip install tensorflow==2.12.0 tensorflow-gpu==2.12.0 opencv-python mediapipe sklearn matplotlib
# install dlib

In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [3]:
# chay 1
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import math

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.chdir("/content/drive/MyDrive/4_Geometric Modelling - Web Mining")

In [6]:
# !unzip "Dataset.zip"

# 2. Keypoints using MP Holistic

In [7]:
# chay 2
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [8]:
# chay 3
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [9]:
# chay 4
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [10]:
# chay 5
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

# 3. Extract Keypoint Values

In [11]:
# chay 6
def extract_keypoints_reduce(results):
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark[:25]]).flatten() if results.pose_landmarks else np.zeros(25*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [12]:
def extract_keypoints_reduce_old(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark[:25]]).flatten() if results.pose_landmarks else np.zeros(25*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

# 5. Collect Keypoint Values for Training and Testing

In [13]:
import os

VIDEO_PATH = os.path.join('Dataset')

# Kiểm tra xem đường dẫn có tồn tại không
if os.path.exists(VIDEO_PATH):
    # Lấy danh sách các tên thư mục con
    subdirectories = [d for d in os.listdir(VIDEO_PATH) if os.path.isdir(os.path.join(VIDEO_PATH, d))]
    print("Danh sách các thư mục con trong", VIDEO_PATH, ":", subdirectories)
else:
    print("Đường dẫn", VIDEO_PATH, "không tồn tại.")

Danh sách các thư mục con trong Dataset : ['zoomIn', 'rotateYN', 'rotateXC', 'moveNear', 'rotateYC', 'zoomOut', 'rotateXN', 'moveTop', 'rotateZN', 'moveRight', 'rotateZC', 'moveBottom', 'moveFar', 'moveLeft']


In [ ]:
# chay 7
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_lh_rh')
VIDEO_PATH = os.path.join('Dataset')
# VIDEO_PATH = os.path.join('Videos')

# Actions that we try to detect 'Correct!', 'hi'
actions = np.array(subdirectories)
# actions = np.array(['moveBottom','move','camthu','congnhan','dau','daubung','daulung','hello','hoabinh','howAre','kham','langnghe','lanh','love','mask','mau','met','metmoi','moico'
# 'no','please','sorry','thanks','wear','you'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # video 2s => 30 frame
            # Đường dẫn tới file video
            video_path = os.path.join(VIDEO_PATH, action, action + '_' + str(sequence))

            # Mở file video
            cap = cv2.VideoCapture(f'{video_path}.avi')

            # Số frame của video
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            print('Tổng số frame: ', total_frames)
            if (total_frames<15):
              continue
            step=1
            # Loop through video length aka sequence length
            for frame_num, i in zip(range(sequence_length), range(0, total_frames, step)):

                # Đọc frame
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # NEW Export keypoints
                keypoints = extract_keypoints_reduce(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

    cap.release()
    cv2.destroyAllWindows()

Tổng số frame:  0
Tổng số frame:  31


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  30
Tổng số frame:  34
Tổng số frame:  30
Tổng số frame:  32
Tổng số frame:  30
Tổng số frame:  31
Tổng số frame:  0
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame:  31
Tổng số frame

# 6. Preprocess Data and Create Labels and Features

In [15]:
# chay 8
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_reduce')

root_dir = 'MP_Data_reduce'
actions = np.array([os.path.basename(dir_name) for dir_name in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, dir_name))])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30
print(actions)

['zoomIn' 'rotateYN' 'rotateXC' 'moveNear' 'rotateYC' 'zoomOut' 'rotateXN'
 'moveTop' 'rotateZN' 'moveRight' 'rotateZC' 'moveBottom' 'moveFar'
 'moveLeft']


In [16]:
# chay 9
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [17]:
# chay 10
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'zoomIn': 0,
 'rotateYN': 1,
 'rotateXC': 2,
 'moveNear': 3,
 'rotateYC': 4,
 'zoomOut': 5,
 'rotateXN': 6,
 'moveTop': 7,
 'rotateZN': 8,
 'moveRight': 9,
 'rotateZC': 10,
 'moveBottom': 11,
 'moveFar': 12,
 'moveLeft': 13}

In [18]:
import os
import shutil

def move_files_if_needed(parent_dir):
    for folder_name in os.listdir(parent_dir):
        folder_path = os.path.join(parent_dir, folder_name)
        if os.path.isdir(folder_path):
            for sub_folder_name in os.listdir(folder_path):
                sub_folder_path = os.path.join(folder_path, sub_folder_name)
                if os.path.isdir(sub_folder_path):
                    files_count = len(os.listdir(sub_folder_path))
                    if files_count < 30:
                        print(files_count)
                        print(sub_folder_path)
                        next_sub_folder_name = str(int(sub_folder_name) + 1)
                        next_sub_folder_path = os.path.join(folder_path, next_sub_folder_name)
                        if os.path.exists(next_sub_folder_path) and len(os.listdir(next_sub_folder_path))>=30:
                            files_to_move = os.listdir(next_sub_folder_path)
                            for file_name in files_to_move:
                                file_path = os.path.join(next_sub_folder_path, file_name)
                                dest_file_path = os.path.join(sub_folder_path, file_name)
                                shutil.copy2(file_path, dest_file_path)
                        else:
                            print("Không có thư mục tiếp theo sau", sub_folder_name)
                        # if os.path.exists(next_sub_folder_path):
                        #     files_to_move = os.listdir(next_sub_folder_path)
                        #     for file_name in files_to_move:
                        #         file_path = os.path.join(next_sub_folder_path, file_name)
                        #         shutil.move(file_path, sub_folder_path)
                        # else:
                        #     print("Không có thư mục tiếp theo sau", sub_folder_name)

# Thay 'Data_train' bằng đường dẫn tới thư mục cha của bạn
parent_directory = 'MP_Data_lh_rh'
move_files_if_needed(parent_directory)

0
MP_Data_lh_rh/zoomIn/0
Không có thư mục tiếp theo sau 0
22
MP_Data_lh_rh/zoomIn/1
Không có thư mục tiếp theo sau 1
0
MP_Data_lh_rh/zoomIn/2
Không có thư mục tiếp theo sau 2
0
MP_Data_lh_rh/zoomIn/3
Không có thư mục tiếp theo sau 3
0
MP_Data_lh_rh/zoomIn/4
Không có thư mục tiếp theo sau 4
0
MP_Data_lh_rh/zoomIn/5
Không có thư mục tiếp theo sau 5
0
MP_Data_lh_rh/zoomIn/6
Không có thư mục tiếp theo sau 6
0
MP_Data_lh_rh/zoomIn/7
Không có thư mục tiếp theo sau 7
0
MP_Data_lh_rh/zoomIn/8
Không có thư mục tiếp theo sau 8
0
MP_Data_lh_rh/zoomIn/9
Không có thư mục tiếp theo sau 9
0
MP_Data_lh_rh/zoomIn/10
Không có thư mục tiếp theo sau 10
0
MP_Data_lh_rh/zoomIn/11
Không có thư mục tiếp theo sau 11
0
MP_Data_lh_rh/zoomIn/12
Không có thư mục tiếp theo sau 12
0
MP_Data_lh_rh/zoomIn/13
Không có thư mục tiếp theo sau 13
0
MP_Data_lh_rh/zoomIn/14
Không có thư mục tiếp theo sau 14
0
MP_Data_lh_rh/zoomIn/15
Không có thư mục tiếp theo sau 15
0
MP_Data_lh_rh/zoomIn/16
Không có thư mục tiếp theo sau 16

In [21]:
def getPoint(res):
  print(res)
  return res

In [ ]:
# chay 11
print("no_sequences:", no_sequences)
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(getPoint(res))
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
# chay 12
X = np.array(sequences)

In [ ]:
# chay 13
y = to_categorical(labels).astype(int)

In [ ]:
# chay 14
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print(X_train.shape)
print(X_test.shape)

# 7. Build and Train LSTM Neural Network

In [ ]:
# chay 15
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, GRU
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# chay 16
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# chay 17
model = Sequential()
model.add(GRU(64, return_sequences=True, activation='relu', input_shape=(30, 226)))
model.add(GRU(128, return_sequences=True, activation='relu'))
model.add(GRU(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# chay 18
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# chay 20
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 30, 64)            56064     
                                                                 
 gru_4 (GRU)                 (None, 30, 128)           74496     
                                                                 
 gru_5 (GRU)                 (None, 128)               99072     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 14)                462       
                                                                 
Total params: 240430 (939.18 KB)
Trainable params: 240

In [ ]:
# chay 19
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
18/18 [==============================] - 5s 212ms/step - loss: 2.5621 - categorical_accuracy: 0.0768
Epoch 2/100
18/18 [==============================] - 3s 150ms/step - loss: 2.2246 - categorical_accuracy: 0.1696
Epoch 3/100
18/18 [==============================] - 2s 137ms/step - loss: 1.7137 - categorical_accuracy: 0.4000
Epoch 4/100
18/18 [==============================] - 4s 204ms/step - loss: 1.2631 - categorical_accuracy: 0.5786
Epoch 5/100
18/18 [==============================] - 4s 215ms/step - loss: 0.9425 - categorical_accuracy: 0.6750
Epoch 6/100
18/18 [==============================] - 2s 137ms/step - loss: 0.7907 - categorical_accuracy: 0.7214
Epoch 7/100
18/18 [==============================] - 2s 138ms/step - loss: 0.5000 - categorical_accuracy: 0.8232
Epoch 8/100
18/18 [==============================] - 2s 138ms/step - loss: 0.4330 - categorical_accuracy: 0.8429
Epoch 9/100
18/18 [==============================] - 4s 230ms/step - loss: 0.3911 - categorical_

KeyboardInterrupt: 

# 8. Make Predictions

# 9. Save Weights

In [ ]:
# chay 21
model.save('actionGRUThreeLayersReduce.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# chay 22
from tensorflow.keras.models import load_model
model = load_model('actionGRUThreeLayersReduce.h5')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Giả sử bạn đã có model đã được huấn luyện và X_test, y_test

# Bước 1: Dự đoán kết quả trên tập test
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Nếu là bài toán phân loại nhiều lớp
y_true = np.argmax(y_test, axis=1)  # Nếu y_test cũng là one-hot encoded

# Bước 2: Tạo ma trận nhầm lẫn
cm = confusion_matrix(y_true, y_pred_classes)

# Bước 3: In và/hoặc vẽ ma trận nhầm lẫn
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

# 10. Evaluation using Confusion Matrix and Accuracy

# 11. Test in Real Time

In [ ]:
# chay 23
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints_reduce(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
#             if (actions[np.argmax(res)]!='nothing'):
#                 sequence = sequence[-15:]


        #3. Viz logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)

#         cv2.rectangle(image, (0,0), (600, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import time

# Hàm để vẽ số đếm lên frame
def draw_count(frame, count):
    cv2.putText(frame, f"Count: {count}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Hàm callback để xử lý dữ liệu từ camera và hiển thị frame
def process_and_display_frame(data):
    for img_data in data:
        frame = cv2.imdecode(np.frombuffer(b64decode(img_data.split(',')[1]), np.uint8), -1)
        # Ghi số đếm lên frame
        process_and_display_frame.count += 1
        draw_count(frame, process_and_display_frame.count)
        # Hiển thị frame
        cv2_imshow(frame)

# Khởi tạo biến đếm
process_and_display_frame.count = 0

# Hàm để chụp ảnh từ camera và gửi dữ liệu đến Python
def capture_and_send_photos(quality=0.8):
    js = Javascript('''
        async function capturePhotos(quality) {
            const photos = [];
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            // Capture 30 photos per second for 1 second
            for (let i = 0; i < 30; i++) {
                const video = document.createElement('video');
                video.style.display = 'none';
                const div = document.createElement('div');
                div.appendChild(video);
                document.body.appendChild(div);
                video.srcObject = stream;
                await video.play();

                // Wait for video to play
                await new Promise(resolve => video.onplaying = resolve);

                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                canvas.getContext('2d').drawImage(video, 0, 0);
                const photo = canvas.toDataURL('image/jpeg', quality);
                photos.push(photo);

                div.remove();
                stream.getVideoTracks()[0].stop();
            }

            // Send photos to Python
            google.colab.kernel.invokeFunction('notebook.capturePhotos', [photos], {});
        }
    ''')
    display(js)

# Bắt đầu chụp ảnh từ camera và gửi dữ liệu đến Python
capture_and_send_photos()

# Hàm callback để nhận dữ liệu từ JavaScript và xử lý
def receive_photos(photos):
    process_and_display_frame(photos)

# Đăng ký callback để nhận dữ liệu từ JavaScript
output.register_callback('notebook.capturePhotos', receive_photos)


<IPython.core.display.Javascript object>

NameError: name 'output' is not defined

In [ ]:
stream.stop()
cv2.destroyAllWindows()

NameError: name 'stream' is not defined

In [ ]:
cap.release()
cv2.destroyAllWindows()